Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn #Neural Networks
import torch.optim as optim #Optimization

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

#Lembrando que nunca usamos todos os dados, devemos dividir o dataframe em um conjunto treino e outro teste
df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler() # Normalizando para um valor entre 0 e 1
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module): # Herdando do nn.Module
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets # Definimos a arquitetura da rede (quantos dados, numero de neuronios nas camadas 1 e 2, e o numero de alvos)
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )
        
        # ReLU: Função retificadora parcialmente linear, onde valores negativos estão em 0 ou próximos, e a identidade para números positivos

    def forward(self, x): # Recebe a informação e a passa por todas as camadas
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1] # Shape diz quantas colunas tem o X_treino
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.0655,  0.0306,  0.1374, -0.3007,  0.0098,  0.2235],
        [ 0.3977,  0.2027, -0.3849,  0.2204, -0.3041,  0.1631],
        [-0.3613,  0.0399, -0.0552, -0.3130, -0.2436,  0.2747],
        [-0.2903, -0.2743, -0.1440, -0.2664,  0.1795, -0.0745],
        [ 0.3317, -0.1519, -0.2617, -0.3352,  0.2517,  0.2541],
        [-0.3985, -0.1907,  0.2006, -0.2237,  0.0241, -0.2466],
        [-0.2962, -0.1609,  0.2766,  0.1696, -0.1142,  0.0582],
        [-0.3022, -0.3557, -0.2369, -0.1907,  0.0565,  0.0842],
        [ 0.2964,  0.2198,  0.1420, -0.1140,  0.3445,  0.2048],
        [ 0.1621,  0.1440,  0.3056, -0.2030,  0.1060, -0.1805],
        [ 0.0441, -0.2675,  0.2635, -0.1940,  0.0718, -0.3977],
        [ 0.2235,  0.2842,  0.0670, -0.2227, -0.3852, -0.0610],
        [-0.0961, -0.3266, -0.3592, -0.2852, -0.1115,  0.2702],
        [ 0.1751,  0.1923,  0.1662, -0.2962, -0.2673, -0.3828],
        [ 0.2769,  0.2506,  0.0322,  0.0038,  0.1526, -0.1353],
        [ 0.2653, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[-0.0676],
        [-0.0656],
        [-0.0658],
        ...,
        [-0.0620],
        [-0.0708],
        [-0.0696]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO) # Adam: Varia a taxa de aprendizado automaticamente

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train() # Não estamos treinando, mas sim colocando a rede em modo Treino

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
# Aqui sim inicia-se o ciclo de treino da rede

NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.1108)
1 tensor(0.1041)
2 tensor(0.0979)
3 tensor(0.0921)
4 tensor(0.0868)
5 tensor(0.0821)
6 tensor(0.0781)
7 tensor(0.0743)
8 tensor(0.0709)
9 tensor(0.0677)
10 tensor(0.0646)
11 tensor(0.0618)
12 tensor(0.0592)
13 tensor(0.0567)
14 tensor(0.0545)
15 tensor(0.0525)
16 tensor(0.0507)
17 tensor(0.0490)
18 tensor(0.0474)
19 tensor(0.0460)
20 tensor(0.0448)
21 tensor(0.0438)
22 tensor(0.0429)
23 tensor(0.0421)
24 tensor(0.0415)
25 tensor(0.0410)
26 tensor(0.0406)
27 tensor(0.0403)
28 tensor(0.0401)
29 tensor(0.0399)
30 tensor(0.0398)
31 tensor(0.0397)
32 tensor(0.0398)
33 tensor(0.0398)
34 tensor(0.0398)
35 tensor(0.0398)
36 tensor(0.0397)
37 tensor(0.0396)
38 tensor(0.0395)
39 tensor(0.0393)
40 tensor(0.0391)
41 tensor(0.0389)
42 tensor(0.0387)
43 tensor(0.0385)
44 tensor(0.0383)
45 tensor(0.0380)
46 tensor(0.0377)
47 tensor(0.0374)
48 tensor(0.0372)
49 tensor(0.0369)
50 tensor(0.0367)
51 tensor(0.0365)
52 tensor(0.0362)
53 tensor(0.0360)
54 tensor(0.0358)
55 tensor(0.0356)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [17]:
with torch.no_grad(): # Operações calculadas normalmente sem usar gradiente. Bom para testar a rede com relação à previsão.
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1542.8842597]
[1409.99999118] [1561.89927435]
[1128.99997044] [1142.64647913]
[12991.99967575] [11714.67422485]
[14720.0001564] [14829.51839447]
[2938.99994564] [2709.16747665]
[6635.00014687] [7969.9736805]
[11215.99973297] [9472.35276604]
[1040.99999189] [1438.53815413]
[5040.00003624] [5246.17535973]
[9541.99988747] [8457.15846443]
[3792.99988461] [5257.15242958]
[1002.0000155] [814.45474577]
[2542.00005341] [1495.54709291]
[6788.00001717] [6474.03068352]
[9386.00025749] [8230.2416935]
[1245.99996853] [1722.32523108]
[997.00000262] [677.4527669]
[708.00000262] [811.85517454]
[1435.99997544] [1502.00853777]
[2590.00006676] [2529.9358325]
[1392.99997497] [1772.64589643]
[11755.99946976] [9828.86610031]
[2007.99997377] [1357.4233737]
[2707.99998474] [2561.12903357]
[631.00000417] [629.04856873]
[652.99999881] [978.08386374]
[2492.00006247] [2579.97151518]
[594.99999416] [630.03815317]
[760.0000056] [1168.46442175]
[1179.00003028] [1151.16123247]
[1789.9999361] [1850.91

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [18]:
minha_MLP.eval() # Modo de avaliação (trocamos do modo train)

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [19]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [880.85956907]
[628.00000334] [613.1620245]
[442.00000328] [827.46031904]
[2284.99997044] [2522.46344328]
[1040.99999189] [836.01613855]
[809.99999654] [1253.33220911]
[3049.99987316] [3548.19507599]
[720.00000596] [619.33159637]
[877.99999249] [515.74108076]
[1292.00000429] [1718.6990056]
[7056.00004578] [4927.95333481]
[4176.99999142] [5882.28867531]
[13701.00020599] [14872.35619736]
[2948.0000515] [4004.19823456]
[16732.99985504] [14534.10876465]
[1815.00000048] [2041.42893839]
[16220.00016022] [11986.78913879]
[5967.00019073] [4747.80293846]
[473.00000042] [589.88618803]
[1078.9999795] [1043.22914076]
[452.00000319] [619.17422199]
[801.99998283] [865.07748175]
[2458.99994993] [2464.66625452]
[2492.99998236] [2660.19455338]
[582.00000203] [756.1027503]
[1405.9999671] [1511.28866529]
[8715.99991035] [11468.60463333]
[7634.00004578] [6459.61948967]
[506.99999839] [686.91231775]
[665.99999094] [800.18301153]
[1815.00000048] [1695.7471509]
[707.00001383] [572.3057332]
[59

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [20]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

# O que o valor de Loss nos diz? Em geral, a loss sempre diminui, mas há o problema de overfitting caso hajam muitas épocas. Uma estrategia para escolher o numero de épocas é usar um conjunto de validação.

# Pegamos uma quantidade teste de um dataset (10%), o que sobrou é o treino. Do que sobrou, podemos fatiar novamente, pegar mais 10% e chamar de validação.

# Ao final de cada época (atualizar parâmetros), pegamos a rede e pedimos para ela rodar nos dados de validação (dados que não passaram pela rede!).

# Fazer uma previsão de perda dos dados de validação, resultando em um loss de validação.

# Logo os dados de vlaidação são o termômetro da rede, pois os dados de validação estando fora n fizeram bakcpropag, e por serem dados que a rede nao conhece, ela não pode prever e então a curva loss sobre epocas começa a decolar.

Loss do teste: 1378.4299159014176


## Conclusão



## Playground

